# Visualization of CIFAR10 results with WideResNets

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve
from membership_testers.utils import get_rmia_score_arrays_from_loss_arrays
from visualize import plot_multiple_roc_curves

# Setting up

In [ ]:
split_matrices = [
    np.load("mi_lira_2021/exp/cifar10/experiment-{}_16/keep.npy".format(i)).reshape(
        1, -1
    )
    for i in range(16)
]

In [ ]:
split_matrix = np.concatenate(split_matrices, axis=0)

In [ ]:
from pathlib import Path
import numpy as np

BASE_DIR = Path("./mi_lira_2021/exp/cifar10")
LOSS_FILE = "loss/0000000100.npy"

# Load losses into a list -------------------------------------------
losses = [
    np.load(BASE_DIR / f"experiment-{i}_16" / LOSS_FILE).squeeze(1)
    for i in range(16)
]

In [ ]:
rmia_offline = get_rmia_score_arrays_from_loss_arrays(
    losses,
    split_matrix,
    offline=True,
    a=0.33
)

In [ ]:
rmia_offline.shape

## No Range

In [ ]:
fpr, tpr, _ = roc_curve(split_matrix[0], rmia_offline[:, 0])
plot_multiple_roc_curves([fpr], [tpr], "Aug, original images")

## Range

In [ ]:
def get_topk(array, k):
    id = (-array).argsort(axis=1)[:, :k]
    return array[np.arange(array.shape[0])[:, None], id]

def get_bottomk(array, k):
    id = array.argsort(axis=1)[:, :k]
    return array[np.arange(array.shape[0])[:, None], id]

### bottomk

In [ ]:
fprs = []
tprs = []

fpr, tpr, _ = roc_curve(split_matrix[0], get_bottomk(rmia_offline, 4).mean(1))
plot_multiple_roc_curves(fprs, tprs, "Aug, bottom-4")